# Introduction 

In this data analysis I will try to figure our who is the most valuable player based on their influence into winning, stats in comparison to their salary. 

Based on that we will be able to figure out which player is Overpaid and which is Underpaid.


In [1]:
#First we need to import Beautiful Soup library that will allow us to do web scraping as well as pandas that will allow us to
#work with our dataframe later on.
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

# Data scraping

In [2]:
r = requests.get('https://www.basketball-reference.com/leagues/NBA_2020_advanced.html')
soup = bs(r.content)

print(str(soup)[0:500])

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202010221" rel="dns-prefetch"/>
<title>2019-20 NBA Player Stats: Advanced | Basketball-Reference.com</title>
<meta conte


As we can see this is not the most beautiful way to show this data. We have to change it!

In [3]:
print(str(soup.prettify())[0:500])

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
  <link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202010221" rel="dns-prefetch"/>
  <title>
   2019-20 NBA Player Stats: Advanced | Basketball-Reference.com
  </


Still not perfect but we can at least distinguish some parts of our html code.

Now it is time to change our HTML code to dataframe using Pandas and BeautifulSoup

In [4]:
#First we need to find our headers for the dataframe
soup.findAll('tr', limit=2)
headers = [th.getText() 
           for th in soup.findAll('tr', limit=2)[0].findAll('th')]

headers = headers[1:]
headers

['Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'MP',
 'PER',
 'TS%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 '\xa0',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 '\xa0',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP']

In [5]:
#Great! Now we need to scrape data of each player row by row using loop
rows = soup.findAll('tr')[1:]
players = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

In [6]:
#Now it is time to input our data into each row using Pandas
stats = pd.DataFrame(players, columns = headers)
stats.head(10)

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
0,Steven Adams,C,26,OKC,63,1680,20.5,.604,.006,.421,...,,3.8,2.7,6.5,.185,,1.9,1.1,2.9,2.1
1,Bam Adebayo,PF,22,MIA,72,2417,20.3,.598,.018,.484,...,,4.6,3.9,8.5,.168,,1.4,2.0,3.4,3.3
2,LaMarcus Aldridge,C,34,SAS,53,1754,19.7,.571,.198,.241,...,,3.0,1.4,4.5,.122,,1.8,-0.5,1.4,1.5
3,Kyle Alexander,PF,23,MIA,2,13,4.7,.500,.000,.000,...,,0.0,0.0,0.0,-0.003,,-6.1,-3.5,-9.6,0.0
4,Nickeil Alexander-Walker,SG,21,NOP,47,591,8.9,.473,.500,.139,...,,-0.7,0.4,-0.2,-0.020,,-3.2,-1.4,-4.6,-0.4
5,Grayson Allen,SG,24,MEM,38,718,12.0,.609,.562,.179,...,,0.8,0.4,1.2,.082,,-0.1,-1.2,-1.3,0.1
6,Jarrett Allen,C,21,BRK,70,1852,20.7,.664,.013,.581,...,,5.2,2.9,8.2,.212,,1.4,1.0,2.3,2.0
7,Kadeem Allen,SG,27,NYK,10,117,14.0,.512,.364,.250,...,,0.1,0.1,0.1,.053,,-0.1,-0.6,-0.7,0.0
8,Al-Farouq Aminu,PF,29,ORL,18,380,7.6,.395,.419,.337,...,,-0.4,0.5,0.1,.014,,-4.4,1.4,-3.0,-0.1
9,Justin Anderson,SF,26,BRK,10,107,3.2,.352,.763,.105,...,,-0.2,0.1,-0.1,-0.054,,-6.8,-0.9,-7.7,-0.2


Amazing we have our dataframe ready but to our later purposes we don't need every column. 
Now it is time to develop our evaluation model which will be used to evaluate how much a player is "worth".

# Modeling

For this project I came up with a formula that in my opinion fits best to the value of each player. It will be weighted average
of a few factors that contribute to player performance and team wins.

My formula is:
    
[(G/Gt)*9+[(50-Age)/50]*4+(Mp/Mt)*9+(PER/15)*9+(TS%/100)*9+[(TRB%+AST%+STL%+BLK%)/100]*5-(TOV%/100)*7+(USG%/100)*7(WS48)*10]/80

The data included here and it's meaning to players value:

G- number of games played. It is very simple if the player is not playing he doesn't add value to a win.

Gt - number of games played in a season which for 2020 is 71

Age - age of a player. If your name is not Lebron James after certain age teams must concider your age when playing back-to-back etc. This contributes to the game plan and season plan

Mp-minutes played. As with games played you add value to the team when you are out on the floor.

Mt - total number of minutes in a season on average (without OT) which is 71*48= 3408

PER - a measeure of Per minute production by a player where average is 15

TS% - true shooting percentage that takes into account every shot that a player took

TRB% - total rebound percentage 

AST% - total assist percentage 

STL% - total steals percentage

BLK% - total blocks percentage

TOV% - total turnovers percentage - I gave this a weight of 7 because every NBA fan knows how deadly lost posessions are 
and even with other stats being perfect it is almost impossible to win with great amount of turnovers 

USG% - usage percentage. An estimate of plays that used specific player in them.

WS/48 - estimate of numbers of wins contributed by a player by 48 minutes.

# Adapting dataframe to our needs

In [7]:
#First we need to drop columns that we will not be using later on.
stats=stats.drop(['Tm', '3PAr','FTr','ORB%','DRB%','\xa0', 'OWS','DWS','WS','\xa0','OBPM','DBPM','BPM','VORP','Pos'],axis=1)
stats.head()


,Player,Age,G,MP,PER,TS%,TRB%,AST%,STL%,BLK%,TOV%,USG%,WS/48
0,Steven Adams,26,63,1680,20.5,.604,19.2,13.2,1.5,3.4,14.2,17.3,.185
1,Bam Adebayo,22,72,2417,20.3,.598,17.0,24.2,1.7,3.8,17.6,21.2,.168
2,LaMarcus Aldridge,34,53,1754,19.7,.571,12.0,11.4,1.0,4.4,7.8,23.4,.122
3,Kyle Alexander,23,2,13,4.7,.500,12.9,0.0,0.0,0.0,33.3,10.2,-0.003
4,Nickeil Alexander-Walker,21,47,591,8.9,.473,7.5,21.1,1.3,1.1,16.1,23.3,-0.020


In [8]:
for column in stats.columns:
    if column == 'Player':
        continue
    else:
        stats[column]=pd.to_numeric(stats[column], errors='coerce')
        
stats.head()

,Player,Age,G,MP,PER,TS%,TRB%,AST%,STL%,BLK%,TOV%,USG%,WS/48
0,Steven Adams,26.0,63.0,1680.0,20.5,0.604,19.2,13.2,1.5,3.4,14.2,17.3,0.185
1,Bam Adebayo,22.0,72.0,2417.0,20.3,0.598,17.0,24.2,1.7,3.8,17.6,21.2,0.168
2,LaMarcus Aldridge,34.0,53.0,1754.0,19.7,0.571,12.0,11.4,1.0,4.4,7.8,23.4,0.122
3,Kyle Alexander,23.0,2.0,13.0,4.7,0.500,12.9,0.0,0.0,0.0,33.3,10.2,-0.003
4,Nickeil Alexander-Walker,21.0,47.0,591.0,8.9,0.473,7.5,21.1,1.3,1.1,16.1,23.3,-0.020


In [9]:
#Great now it is time to calculate our player value by previously developed model. Let's call it PV (Player Value)

game=(stats['G']/71)*9
age=(50-(stats['Age']/50))*2
minutes = (stats['MP']/3408)*9
per = stats['PER']*(8/15)
ts= stats['TS%']*(9/100)
ave = (stats['TRB%']+stats['AST%']+stats['STL%']+stats['BLK%'])*(5/100)
tov=stats['TOV%']*(6/100)
usg = stats['USG%']*(7/100)
ws = stats['WS/48']*10

value = (game + age+minutes +per +ts + ave -tov +usg+ws)/80
stats['PV']=value

In [10]:
#Let's sort players by their PV and reduce duplicated players.
stats=stats.sort_values('PV', ascending = False)
stats.head(50)
stats.shape
stats=stats.drop_duplicates(subset=['Player'], keep='first')
stats.head(20)


,Player,Age,G,MP,PER,TS%,TRB%,AST%,STL%,BLK%,TOV%,USG%,WS/48,PV
12,Giannis Antetokounmpo,25.0,63.0,1917.0,31.9,0.613,22.1,34.2,1.5,3.0,13.2,37.5,0.279,1.709749
266,James Harden,30.0,68.0,2483.0,29.1,0.626,9.4,35.9,2.3,2.1,14.2,36.3,0.254,1.703341
183,Luka Dončić,20.0,61.0,2047.0,27.6,0.585,14.7,45.7,1.5,0.6,14.8,36.8,0.207,1.674923
390,Damian Lillard,29.0,66.0,2474.0,26.9,0.627,6.0,34.4,1.3,0.8,11.0,30.3,0.225,1.674734
347,Nikola Jokić,24.0,73.0,2336.0,24.9,0.605,17.1,35.2,1.8,1.7,15.8,26.6,0.202,1.669012
331,LeBron James,35.0,67.0,2316.0,25.5,0.577,12.4,49.1,1.6,1.4,15.1,31.5,0.204,1.667814
168,Anthony Davis,26.0,62.0,2131.0,27.4,0.610,14.8,15.1,2.0,6.0,10.4,29.3,0.250,1.661713
387,Kawhi Leonard,28.0,57.0,1848.0,26.9,0.589,11.3,26.4,2.6,1.6,10.2,33.0,0.226,1.642979
672,Trae Young,21.0,60.0,2120.0,23.9,0.595,6.5,45.6,1.4,0.3,16.2,34.9,0.133,1.633193
651,Hassan Whiteside,30.0,67.0,2008.0,25.0,0.644,23.6,5.9,0.6,8.4,12.9,19.6,0.204,1.631876


Now it is time to search for data about current players salaries.


In [11]:
r2 = requests.get('https://www.basketball-reference.com/contracts/players.html')
soup2 = bs(r2.content)
#First we need to find our headers for the dataframe
soup2.findAll('tr', limit=2)
headers2 = [th.getText() 
           for th in soup2.findAll('tr', limit=2)[1].findAll('th')]

headers2 = headers2[1:]
headers2



['Player',
 'Tm',
 '2019-20',
 '2020-21',
 '2021-22',
 '2022-23',
 '2023-24',
 '2024-25',
 'Signed Using',
 'Guaranteed']

In [12]:
#Great! Now we need to scrape data of each player row by row using loop
rows = soup2.findAll('tr')[1:]
playersContracts = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

#Now it is time to input our data into each row using Pandas
contracts = pd.DataFrame(playersContracts, columns = headers2)
contracts.head(10)

,Player,Tm,2019-20,2020-21,2021-22,2022-23,2023-24,2024-25,Signed Using,Guaranteed
0,None,None,None,None,None,None,None,None,None,None
1,Stephen Curry,GSW,"$40,231,758","$43,006,362","$45,780,966",,,,Bird Rights,"$129,019,086"
2,Chris Paul,OKC,"$38,506,482","$41,358,814","$44,211,146",,,,Bird Rights,"$79,865,296"
3,Russell Westbrook,HOU,"$38,178,000","$41,006,000","$43,848,000","$46,662,000",,,Bird Rights,"$123,032,000"
4,John Wall,WAS,"$37,800,000","$40,824,000","$43,848,000","$46,872,000",,,Bird Rights,"$122,472,000"
5,James Harden,HOU,"$37,800,000","$40,824,000","$43,848,000","$46,872,000",,,Bird Rights,"$122,472,000"
6,LeBron James,LAL,"$37,436,858","$39,219,565","$41,002,273",,,,Cap Space,"$76,656,423"
7,Kevin Durant,BRK,"$37,199,000","$39,058,950","$40,918,900","$42,778,850",,,Sign and Trade,"$117,176,850"
8,Blake Griffin,DET,"$34,234,964","$36,595,996","$38,957,028",,,,Bird Rights,"$70,830,960"
9,Kyle Lowry,TOR,"$33,296,296","$30,000,000",,,,,Bird Rights,"$63,296,296"


Incredible we have everything that we need for now but we need to delete first column.

In [13]:

contracts=contracts.drop(contracts.index[0])
contracts.shape
contracts.dropna


<bound method DataFrame.dropna of                 Player   Tm      2019-20      2020-21      2021-22  \
1        Stephen Curry  GSW  $40,231,758  $43,006,362  $45,780,966   
2           Chris Paul  OKC  $38,506,482  $41,358,814  $44,211,146   
3    Russell Westbrook  HOU  $38,178,000  $41,006,000  $43,848,000   
4            John Wall  WAS  $37,800,000  $40,824,000  $43,848,000   
5         James Harden  HOU  $37,800,000  $40,824,000  $43,848,000   
..                 ...  ...          ...          ...          ...   
620     William Howard  UTA      $50,000                             
621      Isaiah Taylor  TOR      $50,000                             
622        Tyler Lydon  SAC      $50,000                             
623    Hollis Thompson  SAC      $50,000                             
624         Tyler Ulis  SAC      $50,000                             

         2022-23 2023-24 2024-25 Signed Using    Guaranteed  
1                                 Bird Rights  $129,019,086  
2

In [14]:
contracts=contracts.sort_values('Player', ascending = True)

In [15]:
contracts.head(20)

,Player,Tm,2019-20,2020-21,2021-22,2022-23,2023-24,2024-25,Signed Using,Guaranteed
551,A.J. Hammons,MIA,"$350,087","$350,087",,,,,,"$700,174"
57,Aaron Gordon,ORL,"$19,863,636","$18,136,364","$16,409,091",,,,Bird Rights,"$54,409,091"
358,Aaron Holiday,IND,"$2,239,200","$2,345,640","$3,980,551",,,,1st Round Pick,"$4,584,840"
431,Abdel Nader,OKC,"$1,618,520","$1,752,950",,,,,Cap Space,"$1,618,520"
492,Admiral Schofield,WAS,"$898,310","$1,517,981","$1,782,621",,,,MLE,"$4,198,912"
26,Al Horford,PHI,"$28,000,000","$27,500,000","$27,000,000","$26,500,000",,,Cap Space,"$97,000,000"
147,Al-Farouq Aminu,ORL,"$9,258,000","$9,720,900","$10,183,800",,,,MLE,"$29,162,700"
348,Alec Burks,PHI,"$2,320,044",,,,,,Minimum Salary,"$2,320,044"
501,Alen Smailagić,GSW,"$898,310","$1,517,981","$1,782,621","$1,930,681",,,MLE,"$2,415,891"
305,Alex Caruso,LAL,"$2,750,000","$2,750,000",,,,,Room Exception,"$5,500,000"


In [16]:
#We need to drop columns that we will not be using later on.
contracts=contracts.drop(['Tm', '2020-21','2021-22','2022-23','2023-24','2024-25', 'Signed Using','Guaranteed'],axis=1)
contracts.head()

,Player,2019-20
551,A.J. Hammons,"$350,087"
57,Aaron Gordon,"$19,863,636"
358,Aaron Holiday,"$2,239,200"
431,Abdel Nader,"$1,618,520"
492,Admiral Schofield,"$898,310"


In [17]:
#We need to change the value of contract to integer and drop '$',',','.' characters

contracts['2019-20']=contracts['2019-20'].str.replace(r'\$',"")
contracts['2019-20']=contracts['2019-20'].str.replace(',',"")
contracts['2019-20']=contracts['2019-20'].str.replace('.',"")
contracts['2019-20']=contracts['2019-20'].astype(float)

contracts.head()       


,Player,2019-20
551,A.J. Hammons,350087.0
57,Aaron Gordon,19863636.0
358,Aaron Holiday,2239200.0
431,Abdel Nader,1618520.0
492,Admiral Schofield,898310.0


In [19]:
#Let's add another column "Contract" to the first df 
for player in stats['Player']:
    stats['Contract']=contracts['2019-20']
    
stats.head()

,Player,Age,G,MP,PER,TS%,TRB%,AST%,STL%,BLK%,TOV%,USG%,WS/48,PV,Contract
12,Giannis Antetokounmpo,25.0,63.0,1917.0,31.9,0.613,22.1,34.2,1.5,3.0,13.2,37.5,0.279,1.709749,32742000.0
266,James Harden,30.0,68.0,2483.0,29.1,0.626,9.4,35.9,2.3,2.1,14.2,36.3,0.254,1.703341,3567720.0
183,Luka Dončić,20.0,61.0,2047.0,27.6,0.585,14.7,45.7,1.5,0.6,14.8,36.8,0.207,1.674923,7068360.0
390,Damian Lillard,29.0,66.0,2474.0,26.9,0.627,6.0,34.4,1.3,0.8,11.0,30.3,0.225,1.674734,1948080.0
347,Nikola Jokić,24.0,73.0,2336.0,24.9,0.605,17.1,35.2,1.8,1.7,15.8,26.6,0.202,1.669012,2321735.0


Now we need to divide our Contract by PV and sort and put it into another Column "CV" Contract Value. After that we will sort it and show our top 20 contracts in the league.

In [30]:
stats['CV']=stats['Contract']/stats['PV']
stats=stats.sort_values('CV', ascending = True)
stats.head(20)

,Player,Age,G,MP,PER,TS%,TRB%,AST%,STL%,BLK%,TOV%,USG%,WS/48,PV,Contract,CV
621,Fred VanVleet,25.0,54.0,1928.0,16.3,0.553,5.6,27.7,2.5,0.9,12.3,21.7,0.145,1.546822,50000.0,32324.349492
612,Myles Turner,23.0,62.0,1826.0,14.8,0.566,12.2,5.4,1.2,6.4,11.4,17.9,0.111,1.533058,50752.0,33105.082685
623,Noah Vonleh,24.0,36.0,377.0,15.5,0.622,17.4,10.7,1.4,1.3,16.5,13.9,0.127,1.446433,50000.0,34567.799717
617,Denzel Valentine,26.0,36.0,488.0,13.0,0.511,8.4,14.2,2.6,1.2,9.8,23.3,0.048,1.432930,50752.0,35418.327523
620,Jarred Vanderbilt,20.0,2.0,5.0,11.1,0.532,10.6,0.0,0.0,0.0,0.0,15.5,0.104,1.351120,50000.0,37006.333652
610,Rayjon Tucker,22.0,20.0,161.0,9.7,0.584,6.8,4.4,0.3,0.5,13.1,16.6,0.063,1.361404,50752.0,37279.175273
611,Evan Turner,31.0,19.0,251.0,6.8,0.424,8.2,20.6,1.7,2.7,26.2,16.4,-0.064,1.326152,50752.0,38270.133890
622,Gabe Vincent,23.0,9.0,83.0,1.2,0.297,3.4,10.1,2.9,0.0,2.6,20.2,-0.117,1.275185,50000.0,39210.010352
605,Karl-Anthony Towns,24.0,35.0,1187.0,26.5,0.642,16.8,22.8,1.3,2.9,13.2,28.8,0.204,1.578205,81678.0,51753.726260
608,P.J. Tucker,34.0,72.0,2467.0,8.3,0.559,10.1,6.0,1.4,1.2,13.9,8.7,0.078,1.503109,81678.0,54339.380719
